<a href="https://colab.research.google.com/github/RAVIKUMARBALIJA/tensorflow2_tutorials/blob/main/Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras

In [4]:
from keras.datasets import fashion_mnist

In [5]:
pip install keras-tuner

     |████████████████████████████████| 71kB 3.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=8cce85011538f8ce75569434f9476b30b546ea4516d5094c335b012ba28d7453
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=c88f965b49d44664f712e179f7b2bfb93753dc0dd06f4576faad9ea6bf133ae1
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [6]:
import kerastuner as kt

In [7]:
from kerastuner.tuners import bayesian,RandomSearch,hyperband

In [8]:
(x_train,y_train),(x_test,y_test)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [9]:
x_train=x_train/255.
x_test=x_test/255.

In [10]:
x_train.shape

(60000, 28, 28)

In [36]:
from tensorflow.keras.layers import Dense,Input,Dropout,Flatten,InputLayer
from tensorflow.keras.models import Sequential,Model

In [38]:
model=Sequential()
model.add(InputLayer(input_shape=(28,28)))
model.add(Flatten())
#hp_units=hp.Int('units',min_value=32,max_value=512,step=32)
model.add(Dense(units=500,activation=tf.nn.leaky_relu))
model.add(Dense(10,activation='softmax'))
#hp_learning_rate=hp.choice('learning_rate',values=[0.001,0.002,0.003])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_cross_entropy',metrics=['accuracy'])

In [64]:
def buildModel(hp):
  model=Sequential()
  model.add(InputLayer(input_shape=(28,28)))
  model.add(Flatten())
  hp_units=hp.Int('units',min_value=32,max_value=512,step=32)
  model.add(Dense(units=hp_units,activation=tf.nn.leaky_relu))
  model.add(Dense(10,activation='softmax'))
  hp_learning_rate=hp.Choice('learning_rate',values=[0.001,0.002,0.003])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='sparse_categorical_crossentropy',metrics=['accuracy'])
  return model

In [65]:
tuner = kt.Hyperband(buildModel,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt/oracle.json


In [66]:
early_stopping=tf.keras.callbacks.EarlyStopping(patience=5,monitor='val_loss')

In [67]:
tuner.search(x_train,y_train,epochs=50,validation_split=0.2,callbacks=[early_stopping])

Trial 30 Complete [00h 00m 40s]
val_accuracy: 0.8824999928474426

Best val_accuracy So Far: 0.8840833306312561
Total elapsed time: 00h 10m 55s
INFO:tensorflow:Oracle triggered exit


In [69]:
best_model=tuner.get_best_models(num_models=1)
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

In [56]:
#model.fit()
#model.fit(x=None, y=None, batch_size=None, epochs=1, verbose='auto', callbacks=None, \
#          validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, \
 #         sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None,\ 
  #        validation_batch_size=None, validation_freq=1, max_queue_size=10, workers=1, use_multiprocessing=False)

In [72]:
print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 352 and the optimal learning rate for the optimizer
is 0.001.



In [76]:
best_model[0].summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 352)               276320    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                3530      
Total params: 279,850
Trainable params: 279,850
Non-trainable params: 0
_________________________________________________________________


In [77]:
model=tuner.hypermodel.build(best_hps)
history=model.fit(x_train,y_train,epochs=50,validation_split=0.2)
val_acc_per_epoch=history.history['val_accuracy']

Epoch 1/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5077 - accuracy: 0.8191 - val_loss: 0.4355 - val_accuracy: 0.8479
Epoch 2/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3966 - accuracy: 0.8573 - val_loss: 0.3848 - val_accuracy: 0.8630
Epoch 3/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3573 - accuracy: 0.8690 - val_loss: 0.3774 - val_accuracy: 0.8620
Epoch 4/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3350 - accuracy: 0.8781 - val_loss: 0.3740 - val_accuracy: 0.8575
Epoch 5/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3182 - accuracy: 0.8829 - val_loss: 0.3584 - val_accuracy: 0.8704
Epoch 6/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3050 - accuracy: 0.8872 - val_loss: 0.3526 - val_accuracy: 0.8742
Epoch 7/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2920 - accuracy: 0.8931 - val_loss: 0.3466 - val_accuracy:

In [80]:
import numpy as np

In [86]:
best_epoch=np.argmax(val_acc_per_epoch)+1

In [85]:
val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1

43

In [88]:
hypermodel=tuner.hypermodel.build(best_hps)
history=hypermodel.fit(x_train,y_train,epochs=best_epoch,validation_split=0.2)

Epoch 1/43
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5076 - accuracy: 0.8204 - val_loss: 0.4463 - val_accuracy: 0.8353
Epoch 2/43
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3936 - accuracy: 0.8583 - val_loss: 0.3937 - val_accuracy: 0.8611
Epoch 3/43
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3606 - accuracy: 0.8684 - val_loss: 0.3637 - val_accuracy: 0.8727
Epoch 4/43
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3335 - accuracy: 0.8782 - val_loss: 0.3433 - val_accuracy: 0.8774
Epoch 5/43
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3196 - accuracy: 0.8836 - val_loss: 0.3571 - val_accuracy: 0.8710
Epoch 6/43
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3030 - accuracy: 0.8886 - val_loss: 0.3431 - val_accuracy: 0.8778
Epoch 7/43
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2912 - accuracy: 0.8930 - val_loss: 0.3835 - val_accuracy:

In [ ]:
hyper